In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import statistics
import math
import datetime
import operator
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import os
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


Using TensorFlow backend.


In [13]:
#load the data processed with correlation test but still with missing data
os.chdir("/Users/liweizhang/Columbia_University/cu_semester2/GitHub/ADA-Project-LoVEWtc/Group0/")
data = pd.read_csv("train_final.csv")

In [14]:
data.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,4,13926,NaN,150.0,1,142.0,...,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,2987001,0,86401,29.0,4,2755,404.0,150.0,2,102.0,...,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,2987002,0,86469,59.0,4,4663,490.0,150.0,3,166.0,...,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,2987003,0,86499,50.0,4,18132,567.0,150.0,2,117.0,...,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,2987004,0,86506,50.0,1,4497,514.0,150.0,2,102.0,...,144.0,1,1,123,1,0,1,1,1,954
5,2987005,0,86510,49.0,4,5937,555.0,150.0,3,226.0,...,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,2987006,0,86522,159.0,4,12308,360.0,150.0,3,166.0,...,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,2987007,0,86529,422.5,4,12695,490.0,150.0,3,226.0,...,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,2987008,0,86535,15.0,1,2803,100.0,150.0,3,226.0,...,500.0,1,1,98,1,0,0,1,1,1727
9,2987009,0,86536,117.0,4,17399,111.0,150.0,2,224.0,...,NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [18]:
#multiple imputation to fill missing data
def multiple_linear_fit(df,replicates):
    for columns in list(df.columns):
        #if not isinstance(df[columns][0],str):
        if columns  not in ['dist1' ,'dist2' ,'card2'] and columns[0] !='D':
            df[columns].fillna(df[columns].mode()[0],inplace = True)
    complete_cols_df = df[df.columns[df.isna().sum() == 0].tolist()]
    #complete_cols_df = complete_cols_df.drop( [x for x in complete_cols_df.columns if isinstance(complete_cols_df[x][0],str)],axis = 1 )
    uncomplete_cols = df.columns[df.isna().any()].tolist()
    for col in uncomplete_cols:
        complete_cols_df[col] = df[col]
        known_part = complete_cols_df[complete_cols_df[col].notna()].as_matrix()
        unknown_part = complete_cols_df[complete_cols_df[col].isna()].as_matrix()
        y = known_part[:, -1]
        X = known_part[:,0:complete_cols_df.shape[1]-2]
        
        lrg = LinearRegression()
        #print(X, y)
        lrg.fit(X, y)
        cnt = 0
        predictedAges = lrg.predict(unknown_part[:,0:complete_cols_df.shape[1]-2])
        for cnt in range(0,replicates-1):
            predictedAges += lrg.predict(unknown_part[:,0:complete_cols_df.shape[1]-2])
            cnt += 1
        df.loc[ (df[col].isna()), col ] = predictedAges/replicates
        complete_cols_df[col] = df[col]
    return df

data2 = multiple_linear_fit(data,5)

In [25]:
#No missing data
data3.isnull().values.any() 

False

In [102]:

data_X = data3.loc[:, data3.columns != 'isFraud']
data_y = data3['isFraud']

X_train, X_test, y_train, y_test = train_test_split(
    data_X,
    data_y,
    test_size=0.25,
    random_state=42)

sc = StandardScaler() #feature scaling
  
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

pca = PCA(.85)

In [103]:
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_sample(X_train_pca, y_train)

lr = LogisticRegression()
lr.fit(X_sm, y_sm)
predictions = lr.predict(X_test_pca)

In [104]:
print('F1-score is ',f1_score(y_test,predictions))
print('Precision',metrics.precision_score(y_test,predictions))
print('Recall is ',recall_score(y_test, predictions))
print('AUC is ',roc_auc_score(y_test,predictions))

F1-score is  0.19120985638854168
Precision 0.11056699510295592
Recall is  0.7065032987747408
AUC is  0.7473358199768456


In [105]:
#without smote
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
lr2 = LogisticRegression()
lr2.fit(X_train_pca, y_train)
predictions2 = lr2.predict(X_test_pca)

print('F1-score is ',f1_score(y_test,predictions2))
print('Precision',metrics.precision_score(y_test,predictions2))
print('Recall is ',recall_score(y_test, predictions2))
print('AUC is ',roc_auc_score(y_test,predictions2))

F1-score is  0.282591725214676
Precision 0.8227272727272728
Recall is  0.1705937794533459
AUC is  0.5846118619742666
